In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - PEFT

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_peft.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_peft.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_peft.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI工作台中打开
    </a>（推荐使用Python-3 CPU笔记本）
  </td>
</table>

## 概述

本笔记本展示了如何使用性能高效的微调库（[PEFT](https://github.com/huggingface/peft)）微调模型，并在 Vertex AI 中使用容器进行推理。[PEFT](https://github.com/huggingface/peft)支持各种模型。本笔记本展示了一些模型的示例，如 OpenLLaMA、Falcon-instruct、BERT、RoBERTa-large 和 XLM-RoBERTa-large 等。

### 目标

- 使用 PEFT 对因果语言模型进行微调，并在 Vertex AI 中进行推理，支持的模型如下

| 模型 | LoRA |
| :- | :- |
| [openlm-research/open_llama_3b](https://huggingface.co/openlm-research/open_llama_3b) | 是 |
| [openlm-research/open_llama_7b](https://huggingface.co/openlm-research/open_llama_7b) | 是 |
| [openlm-research/open_llama_13b](https://huggingface.co/openlm-research/open_llama_13b) | 是 |

- 使用 PEFT 对 instruct 模型进行微调，并在 Vertex AI 中进行推理，支持的模型如下

| 模型 | LoRA |
| :- | :- |
| [tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct) | 是 |
| [tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct) | 是 |


- 使用 PEFT 对序列分类模型进行微调，并在 Vertex AI 中进行推理，支持的模型如下

| 模型 | LoRA |
| :- | :- |
| [bert-base-uncase](https://huggingface.co/bert-base-uncased) | 是 |
| [RoBERTa-large](https://huggingface.co/roberta-large) | 是 |
| [XLM-RoBERTa-large](https://huggingface.co/xlm-roberta-large) | 是 |

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

在开始之前

**注意**: Jupyter运行以`!`为前缀的行作为shell命令，并且将以`$`为前缀的Python变量插入到这些命令中。

只在 Colab 上运行以下命令，如果您使用的是 Workbench，则跳过此部分。

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置Google云项目

1. [选择或创建一个Google云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额度，用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个云存储存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

5. [创建一个服务帐号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，并授予`Vertex AI User`和`Storage Object Admin`角色，用于将经过微调的模型部署到Vertex AI端点。

为实验环境设置以下变量。指定的云存储桶（`BUCKET_URI`）应位于指定的区域（`REGION`）。请注意，多区域桶（例如“us”）不被认为与由多区域范围（例如“us-central1”）覆盖的单个区域匹配。

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
BUCKET_URI = ""  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

import os

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
EXPERIMENT_BUCKET = os.path.join(BUCKET_URI, "peft")
DATA_BUCKET = os.path.join(EXPERIMENT_BUCKET, "data")
MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "model")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### 初始化Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### 定义常量

In [ ]:
# The pre-built training and serving docker images.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20231020_0936_RC00"
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-serve:20231108_1540_RC00"

### 定义常见功能

In [ ]:
import os
from datetime import datetime

from google.cloud import aiplatform


def get_job_name_with_datetime(prefix: str):
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    model_name,
    model_id,
    finetuned_lora_model_path,
    service_account,
    task,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
):
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "FINETUNED_LORA_MODEL_PATH": finetuned_lora_model_path,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=PREDICTION_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/peft_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

本节展示如何使用PEFT LoRA对OpenLLaMA进行微调，包括[openlm-research/open_llama_3b](https://huggingface.co/openlm-research/open_llama_3b)，[openlm-research/open_llama_7b](https://huggingface.co/openlm-research/open_llama_7b)，以及[openlm-research/open_llama_13b](https://huggingface.co/openlm-research/open_llama_13b)。## casual语言建模+ PEFT## casual语言建模+ PEFT

### 用LoRA进行微调

设置基本模型ID。

In [ ]:
model_id = "openlm-research/open_llama_3b"  # @param ["openlm-research/open_llama_3b", "openlm-research/open_llama_7b", "openlm-research/open_llama_13b"]

使用Vertex AI SDK创建和运行自定义训练作业，使用Vertex AI Model Garden的训练图像。

这个例子使用数据集[Abirate/english_quotes](https://huggingface.co/datasets/Abirate/english_quotes)。您可以使用来自[huggingface的数据集](https://huggingface.co/datasets)，或者在Cloud Storage中存储的[Vertex文本模型数据集格式](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format)中的自定义JSONL数据集。

为了使微调更加高效，我们在加载用于微调LoRA模型的预训练模型时启用了量化(8位)。微调LoRA模型所需的GPU内存峰值使用量大约为~7G、~10G和~16G，分别用于[openlm-research/open_llama_3b](https://huggingface.co/openlm-research/open_llama_3b)、[openlm-research/open_llama_7b](https://huggingface.co/openlm-research/open_llama_7b)和[openlm-research/open_llama_13b](https://huggingface.co/openlm-research/open_llama_13b)，使用默认训练参数和示例数据集。理论上，open_llama_3b和open_llama_7b可以在1个V100上进行微调，而open_llama_13b可以在1个A100(40G)上进行微调。为了简单起见，我们选择默认使用1个A100(40G)来支持这些模型在本笔记本中的使用。

#### 【可选】使用自定义数据集进行微调

要使用自定义数据集，您应该在下面的 `dataset_name` 提供一个指向遵循[Vertex文本模型数据集格式](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format)的JSONL文件的 `gs://` URI。`template` 参数是可选的。

例如，这是来自示例数据集 `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl` 的一个数据点：

```json
{"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
```

要使用包含 `input_text` 和 `output_text` 字段的这个示例数据集，将 `dataset_name` 设置为 `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`，将 `template` 设置为 `vertex_sample`。要使用自定义数据集字段的高级用法，请参阅[模板示例](https://github.com/tloen/alpaca-lora/blob/main/templates/alpaca.json)，并提供您自己的JSON模板作为 `gs://` URI。

In [ ]:
# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = "Abirate/english_quotes"  # @param {type:"string"}
# Optional. Template name or gs:// URI to a custom template.
template = ""  # @param {type:"string"}

# Worker pool spec.
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
machine_type = "a2-highgpu-1g"
accelerator_type = "NVIDIA_TESLA_A100"
replica_count = 1
accelerator_count = 1

# Setup training job.
job_name = get_job_name_with_datetime("openllama-lora-train")
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)
output_dir = os.path.join(MODEL_BUCKET, job_name)
output_dir_gcsfuse = output_dir.replace("gs://", "/gcs/")

# Pass training arguments and launch job.
train_job.run(
    args=[
        "--task=causal-language-modeling-lora",
        f"--pretrained_model_id={model_id}",
        f"--dataset_name={dataset_name}",
        f"--output_dir={output_dir_gcsfuse}",
        "--lora_rank=16",
        "--lora_alpha=32",
        "--lora_dropout=0.05",
        "--warmup_steps=10",
        "--max_steps=10",
        "--learning_rate=2e-4",
        f"--template={template}",
    ],
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
)

print("Trained models were saved in: ", output_dir)

使用提供图片进行推断

该部分将模型上传到模型注册表并在端点上部署。

模型部署步骤大约需要15分钟才能完成。

使用LoRA权重的[openlm-research/open_llama_3b](https://huggingface.co/openlm-research/open_llama_3b)、[openlm-research/open_llama_7b](https://huggingface.co/openlm-research/open_llama_7b)和[openlm-research/open_llama_13b](https://huggingface.co/openlm-research/open_llama_13b)的峰值GPU内存使用情况分别为约5.3G、8.7G和15.2G，默认设置下。我们在部署中使用V100以保持简单。

In [ ]:
model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="openllama-peft-serve"),
    model_id=model_id,
    finetuned_lora_model_path=output_dir,
    service_account=SERVICE_ACCOUNT,
    task="causal-language-modeling-lora",
)
print("endpoint_name:", endpoint.name)

请注意：模型权重将在部署成功后下载。因此，在上述模型部署步骤成功之后，您需要额外等待5分钟，然后再运行下面的下一个步骤。否则，在发送请求到端点时可能会看到`ServiceUnavailable：503 502：Bad Gateway`错误。

一旦部署成功，您可以使用文本提示向端点发送请求。

In [ ]:
# # Loads an existing endpoint as below.
# endpoint_name = endpoint.name
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)
instances = [
    {
        "prompt": "Hi Google.",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

整理资源

In [ ]:
# Delete custom train jobs.
train_job.delete()

# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()

教学 + PEFT

本节展示了如何使用PEFT LoRA对猎隼指令进行微调，包括[tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct)和[tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct)。

### 使用LoRA进行微调

设置基础模型ID。

In [ ]:
model_id = "tiiuae/falcon-7b-instruct"  # @param ["tiiuae/falcon-7b-instruct", "tiiuae/falcon-40b-instruct"]

使用Vertex AI SDK创建和运行自定义训练作业，并使用Vertex AI Model Garden训练图像。

此示例使用数据集 [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco)。您可以使用来自 huggingface 的数据集 或者存储在 Cloud Storage 中的 Vertex 文本模型数据集格式 自定义的 JSONL 数据集。`template` 参数是可选的。

对于使用默认训练参数和示例数据集分别对 [tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct) 和 [tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct) 进行微调 LoRA 模型时，GPU 的最大内存使用量分别为 ~11G 和 ~34G。理论上，falcon-7b-instruct 可以在 1 个 P100/V100 上进行微调，而 falcon-40b-instruct 可以在 1 个 A100（40G）上进行微调。为了简单起见，我们选择默认使用 1 个 A100（40G）来支持这些模型在本笔记本中使用。

#### [可选] 使用自定义数据集进行微调

要使用自定义数据集，您应该在下面的`dataset_name`中提供一个指向符合 [Vertex 文本模型数据集格式](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format)的 JSONL 文件的 `gs://` URI。

例如，以下是来自示例数据集 `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl` 的一个数据点：

```json
{"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
```

要使用包含 `input_text` 和 `output_text` 字段的此示例数据集，请将`dataset_name`设置为`gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`，并将`template`设置为`vertex_sample`。如需使用自定义数据集字段进行高级用法，请参阅[模板示例](https://github.com/tloen/alpaca-lora/blob/main/templates/alpaca.json)，并提供您自己的 JSON 模板作为 `gs://` URI。

In [ ]:
# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
# Optional. Template name or gs:// URI to a custom template.
template = ""  # @param {type:"string"}

# Worker pool spec.
machine_type = "a2-highgpu-1g"
accelerator_type = "NVIDIA_TESLA_A100"
replica_count = 1
accelerator_count = 1

# Setup training job.
job_name = get_job_name_with_datetime("falcon-instruct-lora-train")
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)
output_dir = os.path.join(MODEL_BUCKET, job_name)
output_dir_gcsfuse = output_dir.replace("gs://", "/gcs/")

# Pass training arguments and launch job.
max_steps = 10
train_job.run(
    args=[
        "--task=instruct-lora",
        f"--pretrained_model_id={model_id}",
        f"--dataset_name={dataset_name}",
        f"--output_dir={output_dir_gcsfuse}",
        "--lora_rank=64",
        "--lora_alpha=16",
        "--lora_dropout=0.1",
        "--warmup_ratio=0.03",
        f"--max_steps={max_steps}",
        "--max_seq_length=512",
        "--learning_rate=2e-4",
        f"--template={template}",
    ],
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
)

print("Trained models were saved in: ", output_dir)

### 使用图像进行推理

这一部分将模型上传到模型注册表中，并部署到端点。

模型部署步骤大约需要15分钟才能完成。

使用LoRA权重的[tiiuae/falcon-7b-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct)和[tiiuae/falcon-40b-instruct](https://huggingface.co/tiiuae/falcon-40b-instruct)的峰值GPU内存使用量分别为约15.5G和38.2G，使用默认设置。我们在部署中使用V100以简化操作。

In [ ]:
# # If deploy finetuned falcon-40b-instruct models, please set
# machine_type = "a2-highgpu-1g",
# accelerator_type = "NVIDIA_TESLA_A100"
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"

model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="falcon-peft-serve"),
    model_id=model_id,
    finetuned_lora_model_path=os.path.join(output_dir, "checkpoint-" + str(max_steps)),
    service_account=SERVICE_ACCOUNT,
    task="instruct-lora",
    machine_type=machine_type,
    accelerator_type=accelerator_type,
)
print("endpoint_name: ", endpoint.name)

注意：模型权重将在部署成功后下载。因此，在上述模型部署步骤成功之后，需要额外等待5分钟，然后再运行下面的下一步。否则，当您向端点发送请求时，可能会看到ServiceUnavailable: 503 502:Bad Gateway错误。

一旦部署成功，您可以使用文本提示向端点发送请求。

例子：

```
人类：什么是汽车？
助手：一辆汽车，或称为汽车，是一种与道路相连的人类运输系统，用于将人们或货物从一个地方移动到另一个地方。这个术语还涵盖了各种车辆，包括摩托艇、火车和飞机。汽车通常有四个轮子，为乘客设有驾驶舱，配备发动机或电动机。汽车自19世纪初就已存在，现在是最流行的交通方式之一，用于日常通勤、购物和其他目的。
```

In [ ]:
# # Loads an existing endpoint as below.
# endpoint_name = endpoint.name
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)
instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

清理资源

In [ ]:
# Delete custom train jobs.
train_job.delete()

# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()

序列分类 + PEFT

本节演示了如何使用PEFT LoRA对序列分类模型进行微调，包括[bert-base-uncased](https://huggingface.co/bert-base-uncased)，[RoBERTa-large](https://huggingface.co/roberta-large)和[XLM-RoBERTa-large](https://huggingface.co/xlm-roberta-large)。

### 使用PEFT进行微调

设置基本模型ID。

In [ ]:
model_id = "xlm-roberta-large"  # @param ["bert-base-uncased", "roberta-large", "xlm-roberta-large"]

使用Vertex AI SDK 来创建和运行使用Vertex AI Model Garden 训练图像的自定义训练作业。

此示例使用数据集[glue](https://huggingface.co/datasets/glue)。

用于微调 [RoBERTa-large](https://huggingface.co/roberta-large) 和 [XLM-RoBERTa-large](https://huggingface.co/xlm-roberta-large) 模型的最大 GPU 内存使用量分别是 ~8.8G 和 ~15G，使用默认训练参数和示例数据集。理论上，RoBERTa-large 可以在 1 个P100/V100 上进行微调，而 XLM-RoBERTa-large 可以在 1 个A100 (40G) 上进行微调。我们选择默认使用 1 个 A100 (40G) 以简化此笔记本中所有这些模型的支持。

In [ ]:
dataset_name = "glue"  # @param {type:"string"}

# Worker pool spec.
# # Please switch to A100 or other powerful machines if you run out of memory.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
replica_count = 1
accelerator_count = 1

# Setup training job.
job_name = get_job_name_with_datetime(os.path.basename(model_id) + "-lora-train")
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)
output_dir = os.path.join(MODEL_BUCKET, job_name)
output_dir_gcsfuse = output_dir.replace("gs://", "/gcs/")

# Pass training arguments and launch job.
train_job.run(
    args=[
        "--task=sequence-classification-lora",
        f"--pretrained_model_id={model_id}",
        f"--dataset_name={dataset_name}",
        f"--output_dir={output_dir_gcsfuse}",
        "--lora_rank=8",
        "--lora_alpha=16",
        "--lora_dropout=0.1",
        "--num_epochs=20",
        "--batch_size=32",
        "--learning_rate=3e-4",
    ],
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
)

print("Trained models were saved in: ", output_dir)

### 使用图像服务进行推理

这部分将模型上传到模型注册表并部署到终端点。

模型部署步骤大约需要15分钟才能完成。

使用默认设置，[RoBERTa-large](https://huggingface.co/roberta-large) 和[XLM-RoBERTa-large](https://huggingface.co/xlm-roberta-large) 的峰值GPU内存使用量分别为约2G和约2.4G，我们在部署中使用V100以便简便。

In [ ]:
model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="sequence-classification-peft-serve"),
    model_id=model_id,
    finetuned_lora_model_path=output_dir,
    service_account=SERVICE_ACCOUNT,
    task="sequence-classification-lora",
)
print("endpoint_name:", endpoint.name)

注意：模型权重将在部署成功后下载。因此，在上述模型部署步骤成功之后和您运行下面的下一个步骤之前，需要额外等待5分钟的时间。否则，当您向端点发送请求时，您可能会看到“ServiceUnavailable: 503 502:Bad Gateway”错误。

一旦部署成功，您就可以使用文本提示向端点发送请求。

In [ ]:
# # Loads an existing endpoint as below.
# endpoint_name = endpoint.name
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)
instances = [
    {"prompt": "The cat sat on the mat."},
]
response = endpoint.predict(instances=instances)
labels = [int(item) for item in response.predictions]
print(labels)

清理资源

In [ ]:
# Delete custom train jobs.
train_job.delete()

# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()